In [2]:
from roboflow import Roboflow
from PIL import Image
import numpy as np
import pandas as pd
import os 
import re 

In [3]:
rf = Roboflow("132cxQxyrOVmPD63wJrV") # api keys are individual, change to your own
project = rf.workspace().project("elephant-seals-project-mark-1")
model = project.version("6").model

loading Roboflow workspace...
loading Roboflow project...


In [4]:
# image paths 
path_to_beach_imgs = "Beach_Images/LS 2.13.23"
beach_imgs_paths = [os.path.join(path_to_beach_imgs, file) for file in os.listdir(path_to_beach_imgs)]

In [ ]:
# extracting clumps and getting rid of overlaps 
clump_imgs_dct = {} # dictionary of clumps. image id will be the key and a list of clumps will be its value. 
num_seals = [] # number of individual seals 

# define confidence levels for seals and clumps
seal_conf_lvl = 0
clump_conf_lvl = 0 

def intersects(seal, clump):
    """Check if seal and clump overlap"""
    seal_x1 = seal['x'] - seal['width'] / 2
    seal_x2 = seal['x'] + seal['width'] / 2
    seal_y1 = seal['y'] - seal['height'] / 2
    seal_y2 = seal['y'] + seal['height'] / 2

    clump_x1 = clump['x'] - clump['width'] / 2
    clump_x2 = clump['x'] + clump['width'] / 2
    clump_y1 = clump['y'] - clump['height'] / 2
    clump_y2 = clump['y'] + clump['height'] / 2

    return not (
        seal_x2 <= clump_x1 or
        seal_x1 >= clump_x2 or
        seal_y2 <= clump_y1 or
        seal_y1 >= clump_y2
    )


for path in beach_imgs_paths:

    image = Image.open(path)

    preds = model.predict(path, confidence=20, overlap=30).json().get('predictions', []) # our preset values of min confidence and overlap, based on vibes 

    seals = [pred for pred in preds if pred['class'] == 'seals' and pred['confidence'] > seal_conf_lvl]
    clumps = [pred for pred in preds if pred['class'] == 'clump' and pred['confidence'] > clump_conf_lvl]

    # getting individual seals 
    filtered_seals = [seal for seal in seals if not any(intersects(seal, clump) for clump in clumps)]
    num_seals.append(len(filtered_seals))

    key = re.sub(r'.*/([A-Za-z]+) (\d+)\.(\d+)\.(\d+)\\(DJI_\d+)\.JPG', r'\1\2\3\4_\5', path)
    
    # getting clumps 
    clump_imgs_dct[key] = [] 
    for clump in clumps:
        clump_x1 = clump['x'] - clump['width'] / 2
        clump_x2 = clump['x'] + clump['width'] / 2
        clump_y1 = clump['y'] - clump['height'] / 2
        clump_y2 = clump['y'] + clump['height'] / 2

        top_left_clump = (clump_x1, clump_y1)
        bottom_right_clump = (clump_x2, clump_y2)

        subimage = image.crop((*top_left_clump, *bottom_right_clump))
        
        clump_imgs_dct[key].append(subimage) 

In [10]:
# Extracting Length, Width and RGB metrics 

keys = []
widths = []
heights = []
avg_r = []
sd_r = []
avg_g = []
sd_g = []
avg_b = []
sd_b = [] 

for key, clump_lst in clump_imgs_dct.items():

    for idx, clump in enumerate(clump_lst): 

        keys.append(f"{key}_clump_{idx+1:04d}")
    
        width, height = clump.size

        widths.append(width)
        heights.append(height)

        img_array = np.array(clump)

        avg_r.append(np.mean(img_array[1, :, :]))
        sd_r.append(np.std(img_array[1, :, :]))
        avg_g.append(np.mean(img_array[:, 1, :]))
        sd_g.append(np.std(img_array[:, 1, :]))
        avg_b.append(np.mean(img_array[:, :, 1]))
        sd_b.append(np.std(img_array[:, :, 1]))

heuristics = pd.DataFrame({'key': keys, 
                          'width': widths, 
                          'height': heights,
                          'avg_r': avg_r, 
                          'sd_r': sd_r, 
                          'avg_g': avg_g,
                          'sd_g': sd_g,
                          'avg_b': avg_b,
                          'sd_b': sd_b
                          })

In [12]:
heuristics.head()

,key,width,height,avg_r,sd_r,avg_g,sd_g,avg_b,sd_b
0,LS21323_DJI_0001_clump_0001,222,160,148.968468,8.861153,150.972917,10.472847,144.783333,35.448822
1,LS21323_DJI_0001_clump_0002,400,244,128.905000,16.282485,133.304645,25.476204,125.685543,27.451590
2,LS21323_DJI_0001_clump_0003,324,314,129.433128,18.470493,148.500000,10.585889,136.547643,32.810271
3,LS21323_DJI_0001_clump_0004,248,160,138.185484,11.643010,132.254167,17.457388,139.239592,29.105456
4,LS21323_DJI_0001_clump_0005,162,238,127.631687,9.237278,120.865546,9.841680,127.147240,25.308670


In [ ]:
# writing heuristics 
heuristics.to_csv('heuristics.csv', index=False) 

In [ ]:
# empty set for recording 
counting_seals = pd.DataFrame({'clump': keys, 
                               'Number of Adult Female Seals': None,
                               'Number of Baby Seals': None, 
                               'Number of Adult Male Seals': None
                               })

In [ ]:
# writing empty set 
counting_seals.to_csv('seals_count.csv', index=False)

In [14]:
# saving clumps as images in a new folder  

output_directory = "clumps" # insert folder to save clumps 

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

for key, clump_lst in clump_imgs_dct.items(): 
    for idx, img in enumerate(clump_lst):
        img_path = os.path.join(output_directory, f"{key}_clump_{idx+1:04d}.tif") 
        img.save(img_path)